# Cryptography in Python

### Lets first look at Symmetric key cryptography

In [2]:
from cryptography.fernet import Fernet
class Encryptor():

    def key_create(self):
        key = Fernet.generate_key()
        return key

    def key_write(self, key, key_name):
        with open(key_name, 'wb') as mykey:
            mykey.write(key)

    def key_load(self, key_name):
        with open(key_name, 'rb') as mykey:
            key = mykey.read()
        return key


    def file_encrypt(self, key, original_file, encrypted_file):
        
        f = Fernet(key)

        with open(original_file, 'rb') as file:
            original = file.read()

        encrypted = f.encrypt(original)

        with open (encrypted_file, 'wb') as file:
            file.write(encrypted)

    def file_decrypt(self, key, encrypted_file, decrypted_file):
        
        f = Fernet(key)

        with open(encrypted_file, 'rb') as file:
            encrypted = file.read()

        decrypted = f.decrypt(encrypted)

        with open(decrypted_file, 'wb') as file:
            file.write(decrypted)


In [1]:
# Lets creat a file and store some text on it and encrypt and decrypt it
with open('file.txt','w') as file:
    file.write('Hello, this is secret message')


In [3]:
# now call our methods to get encrypted and decrypted files
encryptor=Encryptor()

mykey=encryptor.key_create()

encryptor.key_write(mykey, 'mykey.key')

loaded_key=encryptor.key_load('mykey.key')

encryptor.file_encrypt(loaded_key, 'file.txt', 'enc_file.txt')

encryptor.file_decrypt(loaded_key, 'enc_file.txt', 'dec_file.txt')


# Calculating Hash of a file in Python

In [6]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes

def calculate_file_hash(file_path):
    sha256_hash = hashes.Hash(hashes.SHA256())

    with open(file_path, 'rb') as file:
        while chunk := file.read(8192):
            sha256_hash.update(chunk)

# now finalize our hash
    file_hash = sha256_hash.finalize()

    return file_hash

# Make sure you have this file in the same directory
file_path = 'message.txt'
hash_value = calculate_file_hash(file_path)

print(f"SHA-256 Hash of '{file_path}': {hash_value.hex()}")


#The := is known as the "walrus operator,"
# We can also pass complete file at once to calculate its hash

SHA-256 Hash of 'message.txt': c8ed5e65713f8170b8d0769a8ed8c35740ba0b706d5856e22d709be1e001cf8c


# Asymmetric Cryptography in Python

In [8]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes

# generate public and private keys
def generate_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
    )

    # public-private key pair
    public_key = private_key.public_key()

#Before serializing, cryptographic keys are typically represented as objects in memory.
     
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )

    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_pem, public_pem

def encrypt(message, public_key):
    # load our keys
    loaded_public_key = serialization.load_pem_public_key(public_key)

    #Encrypt the message 
    ciphertext = loaded_public_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    return ciphertext

def decrypt(ciphertext, private_key):
    
    loaded_private_key = serialization.load_pem_private_key(private_key, password=None)

    #Decrypt the ciphertext
    decrypted_message = loaded_private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    return decrypted_message



In [9]:
private_key, public_key = generate_keys()

original_message = b'This is a secret message.'

encrypted_message = encrypt(original_message, public_key)
decrypted_message = decrypt(encrypted_message, private_key)

print(f'Original Message: {original_message.decode()}')
print(f'Encrypted Message: {encrypted_message.hex()}')
print(f'Decrypted Message: {decrypted_message.decode()}')

Original Message: This is a secret message.
Encrypted Message: 07c3bfcd507861832603b98682fdfab26769dbb200cdaa72720076053d7ca7ee1758d3a02d5b601e0a2e8d924debe8f124e682873301740855492b174f778a90bee2b2522e0937b79515f9e1f79ba41a16acc50cc70c99c1c4efbf1f19003d9ba44f868a8eddd6ad07cf64f49308ce900a7d36c11ef92399d341082dd717a84a7c57c369000c1f4467759fd54943132a264060718a3a3890d5b4e617446acf709412fe95a0c6b7481b67e45ec0edf6cd3217271c5e5cc0de71712ee791f30e3d5531d573fe95518899eda20d6d3f28f0899cf672a8f5cfd83f4df89d99104dbce3c8e7c51d8e3878fb1e801fd9555a3f7757c7a1dd9ee5a67b7ebe948fec2ce2
Decrypted Message: This is a secret message.
